# Indice

### Contextualização
- [Delimitação do Escopo de Trabalho](#Delimitação-do-Escopo-de-Trabalho)

### Inicialização
- [Importar as bibliotecas](#Importar-as-bibliotecas)
- [Setar Parâmetros](#Setar-Parâmetros)

### Obtenção dos dados 
- [Dados das Despesas dos Municipios](#Dados-das-Despesas-dos-Municípios)
- [Gravação dos dados originais em arquivo externo](#Gravação-dos-dados-originais-em-arquivo-externo)
- [Gravação dos dados originais em DataFrame](#Gravação-dos-dados-originais-em-DataFrame)

### Tratamento dos dados 
- [Retirada de colunas](#Retirada-de-colunas)
- [Conversão dos dados](#Conversão-dos-dados)
- [Limpeza dos dados](#Limpeza-dos-dados)

### Consolidação dos dados 
- [Inclusão de Classificação das Contas: Analíticas e Sintéticas](#Inclusão-de-Classificação-das-Contas:-Analíticas-e-Sintéticas) 
- [Inclusão de Classificação das Contas: Tipo de Gasto](#Inclusão-de-Classificação-das-Contas:-Tipo-de-Gasto) 
- [Gravação dos dados tratados em arquivo externo](#Gravação-dos-dados-tratados-em-arquivo-externo)








# <font color=blue> --------------------------------- Contextualização --------------------------------- </font>
[Voltar](#Indice)

### Delimitação do Escopo de Trabalho
[Voltar](#Indice)

#### `Problema de Pesquisa`: 
A partir de __despesas vinculadas ao MDE__, como identificar __indícios de irregularidade nos gastos públicos em educação básica__, executados pelos municípios, de forma a direcionar os trabalhos de auditoria e fiscalização? 

#### `Objetivo Geral do Trabalho`:
Subsidiar o trabalho de fiscalização dos investimentos públicos em educação básica, utilizando-se técnicas de mineração de dados para a identificação de padrões nas despesas em MDE e, em especial, para a detecção de anomalias (valores discrepantes em relação ao padrão) nos gastos realizados pelos municípios.

Desta forma, restringe-se o __escopo__ do trabalho às seguintes __categorias de despesas__:
- Despesas próprias custeadas com impostos e transferências
- Despesas efetuadas com os recursos do Fundeb


#### `Algumas Perguntas a serem respondidas`:

- Identificar a distribuição dos valores de despesas em cada grupo (despesas próprias, despesas do fundeb, despesas de recursos vinculados), para estados e municípios;
- Identificar as maiores despesas estaduais em MDE por modalidade de ensino;
- Identificar as maiores despesas municipais em MDE por modalidade de ensino;
- Listar os estados em que as despesas com MDE são acima das medias nacionais;
- Listar os municipios em que as despesas com MDE são acima das medias estaduais.


# <font color=blue> --------------------------------- Inicialização --------------------------------- </font>
[Voltar](#Indice)

### Importar as bibliotecas
[Voltar](#Indice)

In [1]:
# Loading packages

import warnings 
warnings.filterwarnings("ignore")
import datetime
#from datetime import datetime
import traceback 
import os
import getpass

# Conexao às bases de dados
import pyodbc 
import pymssql

#Analysis 
import pandas as pd 
import numpy as np 
import csv

#Visualization
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
from pandas.plotting import scatter_matrix

# Transformação de variáveis
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Clusterização
from sklearn.cluster import KMeans

# Modelos 
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Separação dados treino e teste
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import cross_val_predict

# Avaliação
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

### Setar Parâmetros
[Voltar](#Indice)

In [2]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 200
pd.set_option('max_colwidth', 1000) # tamanho das colunas para exibição do conteúdo de textos longos

np.set_printoptions(threshold=1000)

# Formatar numeros float (2 casas decimais)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))

# Formatações para gráficos
sns.set(style="darkgrid") #cor da area dos plots
figsize = (10,10)

random_state = 123

# <font color=blue> ---------------- Obtenção dos dados ---------------- </font>
[Voltar](#Indice)

### Dados das Despesas dos Municípios
[Voltar](#Indice)


Tabelas:
- DESPESAS_MUNIC_proprias_2018
- DESPESAS_MUNIC_FUNDEB_2018
- DESPESAS_MUNIC_VINC_2018


In [ ]:
despesas_mun_2018 = ''' 
SELECT  
	COD_UF as CodUF, NOM_UF as NomeUF, SIG_UF as SigUF,
	COD_MUNI as CodMunicipio, 
	NOM_MUNI as NomeMunicipio,
    Classif_Pasta as GrupoDespesa, 
    Codigo_subfuncao_Pasta as CodSubFuncao,
    'N/A' as NomePrograma,
    CodPasta_PAI as CodPasta_Pai, Past_Pai as NomePasta_Pai,
    Cod_Exib_Pasta as CodPasta, Nome_Pasta as NomePasta,
    Nivel_Pasta as NivelPasta, Ordem_Pasta as OrdPasta, 
    cod_item as CodItem, num_orde as NumItem, num_nive as NivelItem, 
    COD_ITEM_PAI as CodItem_Pai,
    Cod_CC as CodCC, Cod_CC_f as CodCC_f, Nome_Conta_Contabil as NomeCC,
    [Dotação Atualizada] as DA, [Desp. Empenhadas] as DE, [Desp. Liquidadas] as DL, [Desp. Pagas] as DP
FROM dbo.DESPESAS_MUNIC_proprias_2018
UNION ALL
SELECT  COD_UF as CodUF, NOM_UF as NomeUF, SIG_UF as SigUF,
		COD_MUNI as CodMunicipio, 
		NOM_MUNI as NomeMunicipio,
        Classif_Pasta as GrupoDespesa, 
        Codigo_subfuncao_Pasta as CodSubFuncao,
        'N/A' as NomePrograma,
        CodPasta_PAI as CodPasta_Pai, Past_Pai as NomePasta_Pai,
        Cod_Exib_Pasta as CodPasta, Nome_Pasta as NomePasta,
        Nivel_Pasta as NivelPasta, Ordem_Pasta as OrdPasta, 
        cod_item as CodItem, num_orde as NumItem, num_nive as NivelItem, COD_ITEM_PAI as CodItem_Pai,
        Cod_CC as CodCC, Cod_CC_f as CodCC_f, Nome_Conta_Contabil as NomeCC,
        [Dotação Atualizada] as DA, [Desp. Empenhadas] as DE, [Desp. Liquidadas] as DL, [Desp. Pagas] as DP
FROM dbo.DESPESAS_MUNIC_FUNDEB_2018
UNION ALL
SELECT COD_UF as CodUF, NOM_UF as NomeUF, SIG_UF as SigUF,
		COD_MUNI as CodMunicipio, 
		NOM_MUNI as NomeMunicipio,
        Classif_Pasta as GrupoDespesa, 
        Codigo_subfuncao_Pasta as CodSubFuncao,
        Nome_Programa as NomePrograma,
        CodPasta_PAI as CodPasta_Pai, Past_Pai as NomePasta_Pai,
        Cod_Exib_Pasta as CodPasta, Nome_Pasta as NomePasta,
        Nivel_Pasta as NivelPasta, Ordem_Pasta as OrdPasta, 
        cod_item as CodItem, num_orde as NumItem, num_nive as NivelItem, COD_ITEM_PAI as CodItem_Pai,
        Cod_CC as CodCC, Cod_CC_f as CodCC_f, Nome_Conta_Contabil as NomeCC,
        [Dotação Atualizada] as DA, [Desp. Empenhadas] as DE, [Desp. Liquidadas] as DL, [Desp. Pagas] as DP
FROM dbo.DESPESAS_MUNIC_VINC_2018
''' 

In [ ]:
cursor.execute(despesas_mun_2018)

In [ ]:
df_desp_mun = pd.read_sql(despesas_mun_2018,conn)

In [ ]:
conn.close()

### Gravação dos dados originais em arquivo externo
[Voltar](#Indice)

Pressupõe-se a execução do SQL apenas uma vez, pois os dados serão gravados em um arquivo csv para posteriores execuções do caderno.

In [ ]:
# Salvar dados do dataframe em csv
df_desp_mun.to_csv('DadosDespesas_Munic_original.csv', sep=';', index=False)

### Gravação dos dados originais em DataFrame
[Voltar](#Indice)

Pressupõe-se, sempre que necessário, a leitura dos dados do arquivo csv preparado previamente no passo anterior.

In [3]:
df = pd.read_csv('DadosDespesas_Munic_original.csv', sep=';',  low_memory=False) 

# <font color=blue> ---------------- Tratamento dos dados ---------------- </font>
[Voltar](#Indice)



### Retirada de colunas

[Voltar](#Indice)



In [4]:
# Retirar colunas desnecessárias (informações de hierarquia dos itens = contas contábeis)
df_itens = df[['CodItem', 'NumItem', 'NivelItem', 'CodItem_Pai']]
colunas_item = ['CodItem', 'NumItem', 'NivelItem', 'CodItem_Pai']

for c in colunas_item:
    del df[c]
    
print('Total de registros de despesas dos Municípios: ', df.shape[0], 'despesas')
print('Total de colunas do dataframe: ', df.shape[1], 'colunas')
print('\nColunas do dataframe de despesas:', '\n', df.columns.values)

Total de registros de despesas dos Municípios:  2026697 despesas
Total de colunas do dataframe:  21 colunas

Colunas do dataframe de despesas: 
 ['CodUF' 'NomeUF' 'SigUF' 'CodMunicipio' 'NomeMunicipio' 'GrupoDespesa'
 'CodSubFuncao' 'NomePrograma' 'CodPasta_Pai' 'NomePasta_Pai' 'CodPasta'
 'NomePasta' 'NivelPasta' 'OrdPasta' 'CodCC' 'CodCC_f' 'NomeCC' 'DA' 'DE'
 'DL' 'DP']


### Conversão dos dados
[Voltar](#Indice)



In [5]:
colunas_string = ['CodUF','CodMunicipio','CodSubFuncao','CodPasta_Pai','CodPasta','NivelPasta','OrdPasta','CodCC']
for col in colunas_string:
    df[col] = df[col].astype(object)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026697 entries, 0 to 2026696
Data columns (total 21 columns):
CodUF            object
NomeUF           object
SigUF            object
CodMunicipio     object
NomeMunicipio    object
GrupoDespesa     object
CodSubFuncao     object
NomePrograma     object
CodPasta_Pai     object
NomePasta_Pai    object
CodPasta         object
NomePasta        object
NivelPasta       object
OrdPasta         object
CodCC            object
CodCC_f          object
NomeCC           object
DA               float64
DE               float64
DL               float64
DP               float64
dtypes: float64(4), object(17)
memory usage: 324.7+ MB


### Limpeza dos dados
[Voltar](#Indice)



In [6]:
print('\nExemplos de registros com campos nulos (campos NaN):')
display(df[['DE','DL','DP', 'NomePrograma', 'CodPasta_Pai','NomePasta_Pai']].sample(10))

# Verificar campos com valores nulos
print('\nQuantidade/ Percentual de campos Nulos:')

total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(10)


Exemplos de registros com campos nulos (campos NaN):


,DE,DL,DP,NomePrograma,CodPasta_Pai,NomePasta_Pai
879579,33116.60,33116.60,31930.00,NaN,365.00,Educação Infantil (Pré-Escola)
1101278,3489.00,3489.00,3489.00,NaN,361.00,Ensino Fundamental
1332540,nan,nan,nan,Transferências de Convênios - Educação,361.00,Ensino Fundamental
987611,nan,nan,nan,NaN,365.00,Educação Infantil (Pré-Escola)
1608305,57027.90,57027.90,57027.90,PNATE,361.00,Ensino Fundamental
1732952,nan,nan,nan,PNATE,361.00,Ensino Fundamental
365477,253247.90,253247.90,253247.90,NaN,365.00,Educação Infantil (Creche) - Exceto FUNDEB
667209,350000.00,350000.00,350000.00,NaN,363.00,Ensino Profissional (Qualificação para o Trabalho)
1344820,23349.60,23349.60,21593.90,PNAE,361.00,Ensino Fundamental
699850,826.30,826.30,826.30,NaN,361.00,Ensino Fundamental - Exceto FUNDEB



Quantidade/ Percentual de campos Nulos:


,Total,Percent
NomePrograma,1284496,0.63
DP,619389,0.31
DL,600948,0.30
DE,583521,0.29
DA,506046,0.25
CodPasta_Pai,19711,0.01
NomeUF,0,0.00
SigUF,0,0.00
CodMunicipio,0,0.00
NomeMunicipio,0,0.00


In [7]:
# Verificação das contas com o campo CodPasta_Pai nulo

print('Grupo de Despesa com campo CodPasta_Pai nulo: ')
print(df[df['CodPasta_Pai'].isnull()]['GrupoDespesa'].value_counts())

Grupo de Despesa com campo CodPasta_Pai nulo: 
Desp proprias - não EF    19711
Name: GrupoDespesa, dtype: int64


In [8]:
# Verificação mais detalhada dos registros com o campo CodPasta_Pai nulo
# Percebe-se que são registros que não estão classificados em alguma modalidade de ensino 

contas_sem_CodPai = df[df['CodPasta_Pai'].isnull()]\
                            [['GrupoDespesa','NomePasta_Pai','CodPasta','NomePasta','NomeCC']].\
                            groupby(['GrupoDespesa','NomePasta_Pai', 'CodPasta','NomePasta']).count().reset_index()

contas_sem_CodPai.columns = ['GrupoDespesa','NomePasta_Pai', 'CodPasta','NomePasta','Total']

print('Contas sem o Codigo da Pasta Pai:')
display(contas_sem_CodPai)
print('Total de registros: ', contas_sem_CodPai['Total'].sum())

Contas sem o Codigo da Pasta Pai:


,GrupoDespesa,NomePasta_Pai,CodPasta,NomePasta,Total
0,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,242,Assistência ao Portador de Deficiência,729
1,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,243,Assistência à Criança e ao Adolescente,1063
2,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,271,Previdência Básica,165
3,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,272,Previdência do Regime Estatutário (Plano de Seguridade Social do Servidor),817
4,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,273,Previdência Complementar (Complementação de Proventos e Aposentadoria),8
5,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,274,Previdência Especial,11
6,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,392,Difusão Cultural,8961
7,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,695,Turismo,533
8,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,722,Telecomunicações (Educação a Distância),1025
9,Desp proprias - não EF,Despesas Próprias Custeadas com Impostos e Transferências,812,Desporto Comunitário,5066


Total de registros:  19711


#### A função educação apresenta uma série de subfunções, como:
- 242 - Assistência ao Portador de Deficiência
- 243 - Assistência à Criança e ao Adolescente
- 271 - Previdência Básica
- 272 - Previdência do Regime Estatutário
- 361 - Ensino Fundamental
- 362 - Ensino Médio
- 363 - Ensino Profissional
- 364 - Ensino Superior
- 365 - Educação Infantil
- 392 - Difusão Cultural
- etc.  

Conforme apresentação das pastas e subpastas no sistema SIOPE Municipal (figura abaixo), as subfunções __361 a 365__ representam __subfunções típicas da Educação__, e estas se dividem conforme modalidade de ensino (Fundamental, Médio, Superior, etc).

![image.png](./imagens/DespesasProprias_Mun.png)

#### Subfunções de Apoio Administrativo

Nas __subfunções típicas da Educação__, há outras subfunções caracterizadas como __“Apoio Administrativo”__, e estas favorecem o desenvolvimento das atividades escolares, como planejamento e orçamento, administração geral, comunicação social, tecnologia da informação e constante formação de recursos humanos.

![image.png](./imagens/ApoioAdministrativo_Mun.png)

Nota-se que as __Contas sem CodigoPai__ são referentes às __subfunções__ (ex. Assistência ao Portador de Deficiência, Difusão Cultural, Turismo, etc.) que não se encaixam em alguma modalidade de ensino (subfunção 361, 362, 363, 364, 365).

#### Ações:
- 1) Os campos de despesas (__DA, DE, DL, DP__) que estão nulos serão preenchidos com __valor ZERO__;


- 2) O campo __NomePrograma__ se aplica somente para Despesas Vinculadas. Demais registros que se encontram com valor nulo serão atualizados com o valor __"Não se aplica"__;


- 3) O campo __CodPasta_Pai__ com valor nulo será atualizado para o valor __"Não se aplica"__.

In [9]:
# Preencher valores de despesas NULL com zero
df['DA'].fillna(0, inplace=True)
df['DE'].fillna(0, inplace=True)
df['DL'].fillna(0, inplace=True)
df['DP'].fillna(0, inplace=True)

# Preencher valores de Nome de Programa NULL com "Não se aplica"
df['NomePrograma'].fillna('Não se aplica', inplace=True)

# Preencher valores de Codigo da Pasta PAI NULL com "Não se aplica"
df['CodPasta_Pai'].fillna('Não se aplica', inplace=True)

print('Quantidade de campos Nulos após o tratamento:')
display(df[['DA', 'DE','DL','DP','NomePrograma','CodPasta_Pai']].isnull().sum())

Quantidade de campos Nulos após o tratamento:


DA              0
DE              0
DL              0
DP              0
NomePrograma    0
CodPasta_Pai    0
dtype: int64

# <font color=blue> ---------------- Consolidação dos dados ---------------- </font>
[Voltar](#Indice)

### Inclusão de Classificação das Contas: Analíticas e Sintéticas 
[Voltar](#Indice)

As despesas da educação são compostas de contas contábeis __sintéticas__ e __analíticas__.

Contas sintéticas são aquelas em que há sub-contas abaixo destas. 

Exemplo: a conta __"Contratação por Tempo Determinado"__ é uma conta sintética, pois contabiliza a soma de outras sub-contas contábeis hierarquicamente dispostas abaixo dela.

`3.31.90.04.00.00`	Contratação por Tempo Determinado	
    - 3.31.90.04.01.00	Salário Contrato Temporário	
    - 3.31.90.04.02.00	Salário Família	
    - 3.31.90.04.03.00	Adicional Noturno de Contrato Temporário 
    (...)	
    - 3.31.90.04.99.00	Outras Vantagens Contrato Temporário

Desta forma, é necessário indicar no dataframe a classificação das contas em sintéticas e analíticas.


In [10]:
# Arquivo com a classificação de cada conta, validada pelo FNDE
df_classif_contas = pd.read_csv('classificacao_contas_SIOPE.csv', sep=';')

# Excluir coluna Nome e retirar Contas/ Classificacoes duplicadas 
del(df_classif_contas['Nome'])
del(df_classif_contas['Tipo de Gasto'])
df_classif_contas.drop_duplicates(inplace=True)

print('Total de registros de classificação: ', df_classif_contas.shape[0])
print('Amostra de alguns registros: ')
df_classif_contas.head(10)

Total de registros de classificação:  337
Amostra de alguns registros: 


,Conta,Classificação
0,3.30.00.00.00.00,Sintético
1,3.31.00.00.00.00,Sintético
2,3.31.90.00.00.00,Sintético
3,3.31.90.01.00.00,Analítico
4,3.31.90.03.00.00,Analítico
5,3.31.90.04.00.00,Sintético
6,3.31.90.04.01.00,Analítico
7,3.31.90.04.02.00,Analítico
8,3.31.90.04.03.00,Analítico
9,3.31.90.04.05.00,Analítico


In [11]:
# Juntar a coluna Classificação ao dataframe de contas contábeis
df = pd.merge(df, df_classif_contas, how='left', left_on = 'CodCC_f', right_on='Conta', indicator=True)

# Verificação: Confirmar que a classificação tenha ocorrido para todos os registros 
print('Total de registros com classificação realizada:')
display(df['_merge'].value_counts())

del df['_merge']
del df['Conta']

print('Total de registros de despesas dos Municípios: ', df.shape[0], 'despesas')
print('Total de colunas do dataframe: ', df.shape[1], 'colunas')
print('\nColunas do dataframe de despesas:', '\n', df.columns.values)

print('\nExibir alguns registros do dataframe:', '\n')
display(df[['CodCC_f','NomeCC','DE','Classificação']].head())

Total de registros com classificação realizada:


both          2026697
right_only          0
left_only           0
Name: _merge, dtype: int64

Total de registros de despesas dos Municípios:  2026697 despesas
Total de colunas do dataframe:  22 colunas

Colunas do dataframe de despesas: 
 ['CodUF' 'NomeUF' 'SigUF' 'CodMunicipio' 'NomeMunicipio' 'GrupoDespesa'
 'CodSubFuncao' 'NomePrograma' 'CodPasta_Pai' 'NomePasta_Pai' 'CodPasta'
 'NomePasta' 'NivelPasta' 'OrdPasta' 'CodCC' 'CodCC_f' 'NomeCC' 'DA' 'DE'
 'DL' 'DP' 'Classificação']

Exibir alguns registros do dataframe: 



,CodCC_f,NomeCC,DE,Classificação
0,3.33.00.00.00.00,OUTRAS DESPESAS CORRENTES,0.00,Sintético
1,3.33.90.39.00.00,SERVIÇOS DE TERCEIROS - PESSOA JURÍDICA,0.00,Sintético
2,3.44.90.52.34.00,"Máquinas, Utensílios e Equipamentos Diversos",0.00,Analítico
3,3.33.90.30.00.00,MATERIAL DE CONSUMO,27288.90,Sintético
4,3.33.00.00.00.00,OUTRAS DESPESAS CORRENTES,1111329.00,Sintético


### Inclusão de Classificação das Contas: Tipo de Gasto 
[Voltar](#Indice)

Em virtude de um grande número de contas contábeis (mais de 300 tipos de contas), foi criada uma nova coluna no dataframe - "Tipo de Gasto".

Desta forma, __cada conta contábil analítica será classificada em um tipo de gasto mais genérico__, com o intuito de ser possível a consolidação das contas em grandes grupos de tipo de gasto.
    - 1. Remuneração
    - 2. Formação
    - 3. Didático
    - 4. Alimentação
    - 5. Transporte
    - 6. Manutenção
    - 7. Investimentos
    - 8. Conveniadas
    - 9. Outros
    
As contas sintéticas não serão classificadas em algum tipo de gasto.    





In [12]:
# Arquivo com a classificação de cada conta em relação ao tipo de gasto, validada pela auditoria da CGU
df_tipo_gastos = pd.read_csv('classificacao_contas_SIOPE.csv', sep=';')

# Excluir coluna Nome e retirar Contas/ Classificacoes duplicadas 
del(df_tipo_gastos['Nome'])
del(df_tipo_gastos['Classificação'])
df_tipo_gastos.drop_duplicates(inplace=True)

# Preencher valores de Tipo de Gasto NULL com "Não se aplica"
df_tipo_gastos['Tipo de Gasto'].fillna('Não se aplica', inplace=True)

print('Total de registros de classificação: ', df_tipo_gastos.shape[0])

print('\nTotal das contas em cada tipo de gasto:')
display(df_tipo_gastos['Tipo de Gasto'].value_counts().sort_index())

print('Amostra de alguns registros: ')
df_tipo_gastos.head(10)

Total de registros de classificação:  337

Total das contas em cada tipo de gasto:


1. Remuneração       96
2. Formação           6
3. Didático           8
4. Alimentação        4
5. Transporte         3
6. Manutenção       106
7. Investimentos     54
8. Conveniadas       12
9. Outros            19
Não se aplica        29
Name: Tipo de Gasto, dtype: int64

Amostra de alguns registros: 


,Conta,Tipo de Gasto
0,3.30.00.00.00.00,Não se aplica
1,3.31.00.00.00.00,Não se aplica
2,3.31.90.00.00.00,Não se aplica
3,3.31.90.01.00.00,9. Outros
4,3.31.90.03.00.00,9. Outros
5,3.31.90.04.00.00,Não se aplica
6,3.31.90.04.01.00,1. Remuneração
7,3.31.90.04.02.00,1. Remuneração
8,3.31.90.04.03.00,1. Remuneração
9,3.31.90.04.05.00,1. Remuneração


In [13]:
# Juntar a coluna Classificação ao dataframe de contas contábeis
df = pd.merge(df, df_tipo_gastos, how='left', left_on = 'CodCC_f', right_on='Conta', indicator=True)

# Verificação: Confirmar que a classificação tenha ocorrido para todos os registros 
print('Total de registros com classificação realizada:')
display(df['_merge'].value_counts())

del df['_merge']
del df['Conta']

print('\nExibir alguns registros do dataframe:', '\n')
display(df[['CodCC_f','NomeCC','DE','Classificação','Tipo de Gasto']].head())

Total de registros com classificação realizada:


both          2026697
right_only          0
left_only           0
Name: _merge, dtype: int64


Exibir alguns registros do dataframe: 



,CodCC_f,NomeCC,DE,Classificação,Tipo de Gasto
0,3.33.00.00.00.00,OUTRAS DESPESAS CORRENTES,0.00,Sintético,Não se aplica
1,3.33.90.39.00.00,SERVIÇOS DE TERCEIROS - PESSOA JURÍDICA,0.00,Sintético,Não se aplica
2,3.44.90.52.34.00,"Máquinas, Utensílios e Equipamentos Diversos",0.00,Analítico,7. Investimentos
3,3.33.90.30.00.00,MATERIAL DE CONSUMO,27288.90,Sintético,Não se aplica
4,3.33.00.00.00.00,OUTRAS DESPESAS CORRENTES,1111329.00,Sintético,Não se aplica


Abaixo segue um exemplo, do minicípio de Angra dos Reis, de uma pasta "Educação Especial" com uma conta sintética __"Obrigações Patronais"__ e 2 sub-contas (analíticas):
   - FGTS
   - Outras Obrigações Patronais


Percebe-se que a soma das contas analíticas é o total da conta sintética.

In [14]:
df[(df['CodCC_f'].str.contains('3.31.90.13')) \
                       & (df['SigUF']=='RJ') \
                       & (df['NomeMunicipio']=='Angra dos Reis')
                       & (df['NomePasta']=='Educação Especial')] \
                [['SigUF','NomeMunicipio','GrupoDespesa','NomePasta','CodCC_f','NomeCC',\
                  'DE','Classificação','Tipo de Gasto']]. \
                sort_values(by=['SigUF', 'NomeMunicipio', 'NomePasta', 'CodCC_f'])

,SigUF,NomeMunicipio,GrupoDespesa,NomePasta,CodCC_f,NomeCC,DE,Classificação,Tipo de Gasto
373596,RJ,Angra dos Reis,Desp proprias - não EF,Educação Especial,3.31.90.13.00.00,Obrigações Patronais,5326.10,Sintético,Não se aplica
487390,RJ,Angra dos Reis,Desp proprias - não EF,Educação Especial,3.31.90.13.01.00,FGTS,1775.40,Analítico,1. Remuneração
430904,RJ,Angra dos Reis,Desp proprias - não EF,Educação Especial,3.31.90.13.99.00,Outras obrigações patronais,3550.70,Analítico,1. Remuneração


In [16]:
# Contagem das contas em cada grupo
df[df['Classificação']=='Analítico']['Tipo de Gasto'].value_counts().sort_index()

1. Remuneração      264333
2. Formação           4699
3. Didático          33816
4. Alimentação       47067
5. Transporte        39084
6. Manutenção       409755
7. Investimentos     99349
8. Conveniadas        6763
9. Outros            18912
Name: Tipo de Gasto, dtype: int64

In [19]:
counts_tg = df[df['Classificação']=='Analítico']['Tipo de Gasto'].value_counts(ascending=False)
perc_tg = df[df['Classificação']=='Analítico']['Tipo de Gasto'].value_counts(normalize=True)*100

display(pd.DataFrame({'Total registros': counts_tg, 'Total de registros em %': perc_tg}).reset_index())


,index,Total registros,Total de registros em %
0,6. Manutenção,409755,44.36
1,1. Remuneração,264333,28.61
2,7. Investimentos,99349,10.75
3,4. Alimentação,47067,5.10
4,5. Transporte,39084,4.23
5,3. Didático,33816,3.66
6,9. Outros,18912,2.05
7,8. Conveniadas,6763,0.73
8,2. Formação,4699,0.51


### Gravação dos dados tratados em arquivo externo

[Voltar](#Indice)



In [20]:
print('Total de registros de despesas dos Municípios: ', df.shape[0], 'despesas')
print('Total de colunas do dataframe: ', df.shape[1], 'colunas')
print('\nColunas do dataframe de despesas:', '\n', df.columns.values)
print('\nTipos de dados das Colunas do dataframe de despesas:', '\n', df.dtypes, sep='')
print('\nQuantidade de valores únicos em cada coluna:\n', df.nunique(), sep='')

Total de registros de despesas dos Municípios:  2026697 despesas
Total de colunas do dataframe:  23 colunas

Colunas do dataframe de despesas: 
 ['CodUF' 'NomeUF' 'SigUF' 'CodMunicipio' 'NomeMunicipio' 'GrupoDespesa'
 'CodSubFuncao' 'NomePrograma' 'CodPasta_Pai' 'NomePasta_Pai' 'CodPasta'
 'NomePasta' 'NivelPasta' 'OrdPasta' 'CodCC' 'CodCC_f' 'NomeCC' 'DA' 'DE'
 'DL' 'DP' 'Classificação' 'Tipo de Gasto']

Tipos de dados das Colunas do dataframe de despesas:
CodUF             object
NomeUF            object
SigUF             object
CodMunicipio      object
NomeMunicipio     object
GrupoDespesa      object
CodSubFuncao      object
NomePrograma      object
CodPasta_Pai      object
NomePasta_Pai     object
CodPasta          object
NomePasta         object
NivelPasta        object
OrdPasta          object
CodCC             object
CodCC_f           object
NomeCC            object
DA               float64
DE               float64
DL               float64
DP               float64
Classificação

In [21]:
# Salvar dados do dataframe em csv
df.to_csv('DadosDespesas_mun_tratados.csv', sep=';', index=False)